# Pipeline Batch Runner - BASE и BASE+ пайплайны

Этот Notebook позволяет запускать BASE и BASE+ пайплайны для нескольких месяцев подряд с автоматическим обновлением файлов прогнозов.

## Особенности:
- Автоматическое обновление файлов прогнозов после каждого месяца
- Запуск обоих пайплайнов (BASE и BASE+) для каждого месяца
- Детальное логирование процесса
- Автоматическое сохранение результатов

In [ ]:
# Импорты и настройки
import pandas as pd
import numpy as np
import yaml
from datetime import datetime, timedelta
import shutil
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Импорт модулей пайплайнов
from generate_synthetic_base_pipeline import process_article
from generate_synthetic_baseplus_pipeline import process_article as process_article_baseplus

print("Все модули успешно импортированы!")
print(f"Время запуска: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
# Конфигурация пайплайнов
config = {
    'data_path': r'd:\work\nwc\DUMMY DATA.xlsx',  # Путь к исходным данным
    'base_predictions_path': r'd:\work\nwc\results\predict_BASE.xlsx',  # Путь к файлу прогнозов BASE
    'baseplus_predictions_path': r'd:\work\nwc\results\synthetic_base_pipeline_BASE.xlsx',  # Путь к файлу прогнозов BASE+
    'backup_dir': r'd:\work\nwc\backups',  # Директория для бэкапов
    'log_file': r'd:\work\nwc\pipeline_batch_log.txt'
}

# Месяцы для обработки (в формате YYYY-MM)
months_to_process = [
    '2024-01',
    '2024-02', 
    '2024-03',
    '2024-04',
    '2024-05',
    '2024-06'
]

# Создание директории для бэкапов если не существует
os.makedirs(config['backup_dir'], exist_ok=True)

print("Конфигурация загружена:")
print(f"- Исходные данные: {config['data_path']}")
print(f"- BASE predictions: {config['base_predictions_path']}")
print(f"- BASE+ predictions: {config['baseplus_predictions_path']}")
print(f"- Месяцы для обработки: {len(months_to_process)} месяцев")
print(f"- Директория бэкапов: {config['backup_dir']}")
print("- Список месяцев:", months_to_process)

In [ ]:
# Импорт утилит из backend
import sys
sys.path.append(r'd:\work\nwc\backend\app')

from utils.config import load_config
from utils.datahandler import load_and_transform_data
from utils.pipelines import run_base_plus_pipeline, run_base_pipeline
from utils.common import generate_monthly_period, setup_custom_logging
from utils.db_usage import export_pipeline_tables_to_excel, SHEET_TO_TABLE, process_exchange_rate_file

# Константы из tasks.py
CONFIG_PATH = r'd:\work\nwc\backend\config_refined.yaml'
TRAINING_FILES_DIR = r'd:\work\nwc\backend\app\training_files'

print("Утилиты backend успешно импортированы!")
print(f"CONFIG_PATH: {CONFIG_PATH}")
print(f"TRAINING_FILES_DIR: {TRAINING_FILES_DIR}")